<a href="https://colab.research.google.com/github/anuraagbarde/market-strategies/blob/master/public_nifty_intraday_strategies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import the csv
import pandas as pd
df = pd.read_csv('NIFTY_2015.csv')
# print(df)

In [2]:
# Preprocessing dataframe
def convertStringDateToDateObject(nDate: int):
    sDate = str(nDate)
    year = int(sDate[0:4])
    month = int(sDate[4:6])
    date = int(sDate[6:])
    res = {
        "year": year,
        "month": month,
        "date": date
    }
    return res

def extractYear(sDate):
    return convertStringDateToDateObject(sDate)['year']

def extractMonth(sDate):
    return convertStringDateToDateObject(sDate)['month']

def extractDate(sDate):
    return convertStringDateToDateObject(sDate)['date']

df['year'] = df.apply(lambda row: extractYear(row.Date), axis=1)
df['month'] = df.apply(lambda row: extractMonth(row.Date), axis=1)
df['date'] = df.apply(lambda row: extractDate(row.Date), axis=1)

df.drop(['Date', 'Instrument'], axis = 1, inplace = True)

df_global = df.copy()

Use `df = df_global.copy()` at the start of each new strategy

# STRATEGY 1

In [3]:
PROFIT_CUTOFF = 1.00
LOSS_CUTOFF = -3.00

def calculatePercentage(start: float, curr: float)->float:
    # print('movement -> ', (curr-start)*100/start)
    return (curr-start)*100/start

def isProfited(start: float, curr: float)->bool:
    if(calculatePercentage(start, curr) > PROFIT_CUTOFF):
        print("Boom!")
        return True
    else:
        return False

def isLost(start: float, curr: float)->bool:
    if(calculatePercentage(start, curr) < LOSS_CUTOFF):
        print("Noo!")
        return True
    else:
        return False

class Day:
    def __init__(self, _date, _start):
        self.date = _date
        self.start = _start
        self.activity = False

    def setProfit(self):
        self.activity = True
        self.profit = True
        self.loss = False

    def setLoss(self):
        self.activity = True
        self.profit = False
        self.loss = True

    def display(self):
        print("Date", self.date)
        print("activity", self.activity)
        if(self.activity == True):
            print("profit", self.profit)
            print("loss", self.loss)
        print()


In [4]:


STATS_inactive = 0
STATS_profit = 0
STATS_loss = 0

# BOILERPLATE-START

YEAR = 2015
MONTHs = range(1, 12+1)

for mm in MONTHs:
    print("Month", mm)
    df = df_global.copy()
    df = df.where((df['year']== YEAR) & (df['month']== mm))
    df = df.dropna()
    DATEs = list(map(lambda x: int(x), df.date.unique()))
    for dd in DATEs:
        df_today = df.copy()
        df_today = df.where(df['date'] == dd)
        df_today = df_today.dropna()
        # BOILERPLATE-END

        # STRATEGY-START
        today = Day(dd, df_today['Open'].iloc[0])

        for _, row in df_today.iterrows():
            if(isProfited(today.start, float(row['High']))):
                today.setProfit()
                break

            if(isLost(today.start, float(row["Low"]))):
                today.setLoss()
                break

        if(today.activity == True):
            if(today.loss == True):
                STATS_loss +=1
            if(today.profit == True):
                STATS_profit +=1
        elif(today.activity == False):
            STATS_inactive += 1
        # today.display()

        # STRATEGY-END

print("STATS_inactive", STATS_inactive)
print("STATS_profit", STATS_profit)
print("STATS_loss", STATS_loss)
print("Net profit", STATS_profit*PROFIT_CUTOFF + STATS_loss*LOSS_CUTOFF)

Month 1
Boom!
Boom!
Boom!
Month 2
Boom!
Boom!
Boom!
Month 3
Boom!
Boom!
Month 4
Boom!
Boom!
Month 5
Boom!
Boom!
Boom!
Boom!
Boom!
Month 6
Boom!
Boom!
Boom!
Month 7
Boom!
Boom!
Boom!
Boom!
Month 8
Boom!
Noo!
Noo!
Boom!
Month 9
Boom!
Boom!
Boom!
Boom!
Boom!
Month 10
Boom!
Month 11
Boom!
Boom!
Month 12
Boom!
Boom!
STATS_inactive 211
STATS_profit 34
STATS_loss 2
Net profit 28.0
